Generate training and testing dataset

In [ ]:
from Preprocessing.Preprocessor import preprocess,split_dataset


preprocess("./inputdata.czi","./preprocessed",256,256,20,148,True,99.8,3,2)

split_dataset("./preprocessed",100)

Train VM-3D model

In [ ]:
from train import train

train(epoch=0, n_epochs=200, )